In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc gem-suite matplotlib

*Estimación de uso: 3 minutos en un procesador Heron r2 (NOTA: Esto es solo una estimación. Su tiempo de ejecución podría variar.)*

## Contexto

Este tutorial demuestra cómo realizar una transición de fase de Nishimori en un procesador cuántico de IBM&reg;. Este experimento fue descrito originalmente en [*Realizing the Nishimori transition across the error threshold for constant-depth quantum circuits*](https://arxiv.org/abs/2309.02863).

La transición de fase de Nishimori se refiere a la transición entre fases con orden de corto y largo alcance en el modelo de Ising con enlaces aleatorios. En una computadora cuántica, la fase con orden de largo alcance se manifiesta como un estado en el que los qubits están entrelazados a lo largo de todo el dispositivo. Este estado altamente entrelazado se prepara utilizando el protocolo de *generación de entrelazamiento por medición* (GEM, por sus siglas en inglés). Mediante el uso de mediciones a mitad de circuito, el protocolo GEM es capaz de entrelazar qubits a lo largo de todo el dispositivo utilizando circuitos de profundidad constante. Este tutorial utiliza la implementación del protocolo GEM del paquete de software [GEM Suite](https://github.com/qiskit-community/gem-suite).

## Requisitos

Antes de comenzar este tutorial, asegúrese de tener instalado lo siguiente:

- Qiskit SDK v1.0 o posterior, con soporte de [visualización](https://docs.quantum.ibm.com/api/qiskit/visualization)
- Qiskit Runtime v0.22 o posterior ( `pip install qiskit-ibm-runtime` )
- GEM Suite ( `pip install gem-suite` )

## Configuración

In [2]:
import matplotlib.pyplot as plt

from collections import defaultdict

from qiskit_ibm_runtime import QiskitRuntimeService

from qiskit.transpiler import generate_preset_pass_manager

from gem_suite import PlaquetteLattice
from gem_suite.experiments import GemExperiment

## Paso 1: Mapear entradas clásicas a un problema cuántico
El protocolo GEM opera sobre un procesador cuántico cuya conectividad de qubits se describe mediante una red (lattice). Los procesadores cuánticos actuales de IBM utilizan la [red heavy hex](https://www.ibm.com/quantum/blog/heavy-hex-lattice). Los qubits del procesador se agrupan en *plaquetas* según la celda unitaria de la red a la que pertenecen. Dado que un qubit puede pertenecer a más de una celda unitaria, las plaquetas no son disjuntas. En la red heavy hex, una plaqueta contiene 12 qubits. Las plaquetas en sí también forman una red, donde dos plaquetas están conectadas si comparten algún qubit. En la red heavy hex, las plaquetas vecinas comparten 3 qubits.

En el paquete de software GEM Suite, la clase fundamental para implementar el protocolo GEM es `PlaquetteLattice`, que representa la red de plaquetas (la cual es distinta de la red heavy hex). Un `PlaquetteLattice` se puede inicializar a partir de un mapa de acoplamiento de qubits. Actualmente, solo se admiten mapas de acoplamiento heavy hex.

La siguiente celda de código inicializa una red de plaquetas a partir del mapa de acoplamiento de un procesador cuántico de IBM. La red de plaquetas no siempre abarca todo el hardware. Por ejemplo, `ibm_torino` tiene 133 qubits en total, pero la red de plaquetas más grande que cabe en el dispositivo utiliza solo 125 de ellos y comprende un total de 18 plaquetas. Lo mismo se puede observar en dispositivos de IBM Quantum&reg; con diferentes cantidades de qubits.

In [ ]:
# QiskitRuntimeService.save_account(channel="ibm_quantum", token="<YOUR_API_KEYN>", overwrite=True, set_as_default=True)
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
plaquette_lattice = PlaquetteLattice.from_coupling_map(backend.coupling_map)

print(f"Number of qubits in backend: {backend.num_qubits}")
print(
    f"Number of qubits in plaquette lattice: {len(list(plaquette_lattice.qubits()))}"
)
print(f"Number of plaquettes: {len(list(plaquette_lattice.plaquettes()))}")

Number of qubits in backend: 133
Number of qubits in plaquette lattice: 125
Number of plaquettes: 18


You can visualize the plaquette lattice by generating a diagram of its graph representation. In the diagram, the plaquettes are represented by labeled hexagons, and two plaquettes are connected by an edge if they share qubits.

In [7]:
plaquette_lattice.draw_plaquettes()

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/625882a4-faeb-4d96-b441-c989f43c4dea-0.avif" alt="Output of the previous code cell" />

Puede visualizar la red de plaquetas generando un diagrama de su representación en forma de grafo. En el diagrama, las plaquetas se representan mediante hexágonos etiquetados, y dos plaquetas están conectadas por una arista si comparten qubits.

In [8]:
# Get a list of the plaquettes
plaquettes = list(plaquette_lattice.plaquettes())
# Display information about plaquette 0
plaquettes[0]

PyPlaquette(index=0, qubits=[0, 1, 2, 3, 4, 15, 16, 19, 20, 21, 22, 23], neighbors=[3, 1])

![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/625882a4-faeb-4d96-b441-c989f43c4dea-0.avif)

Puede obtener información sobre plaquetas individuales, como los qubits que contienen, utilizando el método `plaquettes`.

In [9]:
plaquette_lattice.draw_qubits()

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/a19d63ce-3572-4081-a008-c1332fbbe303-0.avif" alt="Output of the previous code cell" />

In addition to the qubit labels and the edges indicating which qubits are connected, the diagram contains three additional pieces of information that are relevant to the GEM protocol:
- Each qubit is either shaded (gray) or unshaded. The shaded qubits are "site" qubits that represent the sites of the Ising model, and the unshaded qubits are "bond" qubits used to mediate interactions between the site qubits.
- Each site qubit is labeled either (A) or (B), indicating one of two roles a site qubit can play in the GEM protocol (the roles are explained later).
- Each edge is colored using one of six colors, thus partitioning the edges into six groups. This partitioning determines how two-qubit gates can be parallelized, as well as different scheduling patterns that are likely to incur different amounts of error on a noisy quantum processor. Because edges in a group are disjoint, a layer of two-qubit gates can be applied on those edges simultaneously. In fact, it is possible to partition the six colors into three groups of two colors such that the union of each group of two colors is still disjoint. Therefore, only three layers of two-qubit gates are needed to activate every edge. There are 12 ways to so partition the six colors, and each such partition yields a different 3-layer gate schedule.

Now that you have created a plaquette lattice, the next step is to initialize a `GemExperiment` object, passing both the plaquette lattice and the backend that you intend to run the experiment on. The `GemExperiment` class manages the actual implementation of the GEM protocol, including generating circuits, submitting jobs, and analyzing the data. The following code cell initializes the experiment class while restricting the plaquette lattice to only two of the plaquettes (21 qubits), reducing the size of the experiment to ensure that the noise in the hardware doesn't overwhelm the signal.

In [16]:
gem_exp = GemExperiment(plaquette_lattice.filter([9, 12]), backend=backend)

# visualize the plaquette lattice after filtering
plaquette_lattice.filter([9, 12]).draw_qubits()

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/02357c6e-5c83-4ac0-811d-22602d9f33d5-0.avif" alt="Output of the previous code cell" />

También puede generar un diagrama de los qubits subyacentes que forman la red de plaquetas.

In [12]:
circuits = gem_exp.circuits()
print(f"Total number of circuits: {len(circuits)}")

Total number of circuits: 252


![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/a19d63ce-3572-4081-a008-c1332fbbe303-0.avif)

Además de las etiquetas de los qubits y las aristas que indican qué qubits están conectados, el diagrama contiene tres elementos adicionales de información relevantes para el protocolo GEM:
- Cada qubit está sombreado (gris) o sin sombrear. Los qubits sombreados son qubits de "sitio" que representan los sitios del modelo de Ising, y los qubits sin sombrear son qubits de "enlace" utilizados para mediar las interacciones entre los qubits de sitio.
- Cada qubit de sitio está etiquetado como (A) o (B), indicando uno de los dos roles que un qubit de sitio puede desempeñar en el protocolo GEM (los roles se explican más adelante).
- Cada arista está coloreada usando uno de seis colores, particionando así las aristas en seis grupos. Esta partición determina cómo se pueden paralelizar las compuertas de dos qubits, así como diferentes patrones de programación que probablemente incurran en diferentes cantidades de error en un procesador cuántico ruidoso. Dado que las aristas de un grupo son disjuntas, una capa de compuertas de dos qubits puede aplicarse simultáneamente sobre esas aristas. De hecho, es posible dividir los seis colores en tres grupos de dos colores de manera que la unión de cada grupo de dos colores siga siendo disjunta. Por lo tanto, solo se necesitan tres capas de compuertas de dos qubits para activar todas las aristas. Existen 12 formas de particionar los seis colores de esta manera, y cada una de estas particiones produce un programa de 3 capas de compuertas diferente.

Ahora que ha creado una red de plaquetas, el siguiente paso es inicializar un objeto `GemExperiment`, pasando tanto la red de plaquetas como el backend en el que pretende ejecutar el experimento. La clase `GemExperiment` gestiona la implementación real del protocolo GEM, incluyendo la generación de circuitos, el envío de trabajos y el análisis de los datos. La siguiente celda de código inicializa la clase del experimento restringiendo la red de plaquetas a solo dos de las plaquetas (21 qubits), reduciendo el tamaño del experimento para asegurar que el ruido del hardware no domine la señal.

In [13]:
# Restrict experiment to the first scheduling pattern
gem_exp.set_experiment_options(schedule_idx=0)

# There are less circuits now
circuits = gem_exp.circuits()
print(f"Total number of circuits: {len(circuits)}")

# Print the RZZ angles swept over
print(f"RZZ angles:\n{gem_exp.parameters()}")

Total number of circuits: 21
RZZ angles:
[0.         0.07853982 0.15707963 0.23561945 0.31415927 0.39269908
 0.4712389  0.54977871 0.62831853 0.70685835 0.78539816 0.86393798
 0.9424778  1.02101761 1.09955743 1.17809725 1.25663706 1.33517688
 1.41371669 1.49225651 1.57079633]


![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/02357c6e-5c83-4ac0-811d-22602d9f33d5-0.avif)

Un circuito del protocolo GEM se construye siguiendo estos pasos:
1. Preparar el estado $|+\rangle$ en todos los qubits aplicando una compuerta Hadamard a cada qubit.
2. Aplicar una compuerta $R_{ZZ}$ entre cada par de qubits conectados. Esto se puede lograr usando 3 capas de compuertas. Cada compuerta $R_{ZZ}$ actúa sobre un qubit de sitio y un qubit de enlace. Si el qubit de sitio está etiquetado como (B), entonces el ángulo se fija en $\frac{\pi}{2}$. Si el qubit de sitio está etiquetado como (A), entonces se permite que el ángulo varíe, produciendo diferentes circuitos. Por defecto, el rango de ángulos se establece en 21 puntos igualmente espaciados entre $0$ y $\frac{\pi}{2}$, inclusive.
3. Medir cada qubit de enlace en la base de Pauli $X$. Dado que los qubits se miden en la base de Pauli $Z$, esto se puede lograr aplicando una compuerta Hadamard antes de medir el qubit.

Tenga en cuenta que el artículo citado en la introducción de este tutorial utiliza una convención diferente para el ángulo $R_{ZZ}$, que difiere de la convención utilizada en este tutorial por un factor de 2.

En el paso 3, solo se miden los qubits de enlace. Para entender en qué estado permanecen los qubits de sitio, es instructivo considerar el caso en que el ángulo $R_{ZZ}$ aplicado a los qubits de sitio (A) en el paso 2 es igual a $\frac{\pi}{2}$. En este caso, los qubits de sitio quedan en un estado altamente entrelazado similar al estado GHZ,

$$
\lvert \text{GHZ} \rangle = \lvert 00 \cdots 00 \rangle + \lvert 11 \cdots 11 \rangle.
$$

Debido a la aleatoriedad en los resultados de medición, el estado real de los qubits de sitio podría ser un estado diferente con orden de largo alcance, por ejemplo, $\lvert 00110 \rangle + \lvert 11001 \rangle$. Sin embargo, el estado GHZ se puede recuperar aplicando una operación de decodificación basada en los resultados de medición. Cuando el ángulo $R_{ZZ}$ se reduce desde $\frac{\pi}{2}$, el orden de largo alcance aún se puede recuperar hasta un ángulo crítico, que en ausencia de ruido es aproximadamente $0.3 \pi$. Por debajo de este ángulo, el estado resultante ya no exhibe entrelazamiento de largo alcance. Esta transición entre la presencia y la ausencia de orden de largo alcance es la transición de fase de Nishimori.

En la descripción anterior, los qubits de sitio se dejaron sin medir, y la operación de decodificación puede realizarse aplicando compuertas cuánticas. En el experimento tal como se implementa en GEM Suite, que es el que sigue este tutorial, los qubits de sitio sí se miden, y la operación de decodificación se aplica en un paso de posprocesamiento clásico.

En la descripción anterior, la operación de decodificación puede realizarse aplicando compuertas cuánticas a los qubits de sitio para recuperar el estado cuántico. Sin embargo, si el objetivo es medir inmediatamente el estado, por ejemplo, con fines de caracterización, entonces los qubits de sitio se miden junto con los qubits de enlace, y la operación de decodificación puede aplicarse en un paso de posprocesamiento clásico. Así es como se implementa el experimento en GEM Suite, que es el que sigue este tutorial.

Además de depender del ángulo $R_{ZZ}$ en el paso 2, que por defecto recorre 21 valores, el circuito del protocolo GEM también depende del patrón de programación utilizado para implementar las 3 capas de compuertas $R_{ZZ}$. Como se discutió previamente, existen 12 de estos patrones de programación. Por lo tanto, el número total de circuitos en el experimento es $21 \times 12 = 252$.

Los circuitos del experimento se pueden generar utilizando el método `circuits` de la clase `GemExperiment`.

In [14]:
# Get the circuit at index 5
circuit = circuits[5]
# Remove the final measurements to ease visualization
circuit.remove_final_measurements()
# Draw the circuit
circuit.draw("mpl", fold=-1, scale=0.5)

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/fd57d483-c70b-4ad5-b309-15750ad38bac-0.avif" alt="Output of the previous code cell" />

## Step 2: Optimize problem for quantum hardware execution

Transpiling quantum circuits for execution on hardware typically involves a [number of stages](/docs/guides/transpiler-stages). Typically, the stages that incur the most computational overhead are choosing the qubit layout, routing the two-qubit gates to conform to the qubit connectivity of the hardware, and optimizing the circuit to minimize its gate count and depth. In the GEM protocol, the layout and routing stages are unnecessary because the hardware connectivity is already incorporated into the design of the protocol. The circuits already have a qubit layout, and the two-qubit gates are already mapped onto native connections. Furthermore, in order to preserve the structure of the circuit as the $R_{ZZ}$ angle is varied, only very basic circuit optimization should be performed.

The `GemExperiment` class transparently transpiles circuits when executing the experiment. The layout and routing stages are already overridden by default to do nothing, and circuit optimization is performed at a level that only optimizes single-qubit gates. However, you can override or pass additional options using the `set_transpile_options` method. For the sake of visualization, the following code cell manually transpiles the circuit displayed previously, and draws the transpiled circuit.

In [15]:
# Demonstrate setting transpile options
gem_exp.set_transpile_options(
    optimization_level=1  # This is the default optimization level
)
pass_manager = generate_preset_pass_manager(
    backend=backend,
    initial_layout=list(gem_exp.physical_qubits),
    **dict(gem_exp.transpile_options),
)
transpiled = pass_manager.run(circuit)
transpiled.draw("mpl", idle_wires=False, fold=-1, scale=0.5)

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/e9b99d48-8d33-46b5-bff5-480ab1c1c1f2-0.avif" alt="Output of the previous code cell" />

Para los propósitos de este tutorial, es suficiente considerar un solo patrón de programación. La siguiente celda de código restringe el experimento al primer patrón de programación. Como resultado, el experimento tiene solo 21 circuitos, uno para cada ángulo $R_{ZZ}$ recorrido.

In [10]:
exp_data = gem_exp.run(shots=10_000)

To wait for the results, call the `block_for_results` method of the `ExperimentData` object. This call will cause the interpreter to hang until the jobs are finished.

In [11]:
exp_data.block_for_results()

ExperimentData(GemExperiment, d0d5880a-34c1-4aab-a7b6-c4f58516bc03, job_ids=['cwg12ptmptp00082khhg'], metadata=<5 items>, figure_names=['two_point_correlation.svg', 'normalized_variance.svg', 'plaquette_ops.svg', 'bond_ops.svg'])

La siguiente celda de código dibuja un diagrama del circuito en el índice 5. Para reducir el tamaño del diagrama, se eliminan las compuertas de medición al final del circuito.

In [ ]:
def magnetization_distribution(
    counts_dict: dict[str, int],
) -> dict[str, float]:
    """Compute magnetization distribution from counts dictionary."""
    # Construct dictionary from magnetization to count
    mag_dist = defaultdict(float)
    for bitstring, count in counts_dict.items():
        mag = bitstring.count("0") - bitstring.count("1")
        mag_dist[mag] += count
    # Normalize
    shots = sum(counts_dict.values())
    for mag in mag_dist:
        mag_dist[mag] /= shots
    return mag_dist


# Get counts dictionaries with and without decoding
data = exp_data.data()
# Get the last data point, which is at the angle for the GHZ state
raw_counts = data[-1]["counts"]
# Without decoding
site_indices = [
    i for i, q in enumerate(gem_exp.plaquettes.qubits()) if q.role == "Site"
]
site_raw_counts = defaultdict(int)
for key, val in raw_counts.items():
    site_str = "".join(key[-1 - i] for i in site_indices)
    site_raw_counts[site_str] += val
# With decoding
_, site_decoded_counts = gem_exp.plaquettes.decode_outcomes(
    raw_counts, return_counts=True
)

# Compute magnetization distribution
raw_magnetization = magnetization_distribution(site_raw_counts)
decoded_magnetization = magnetization_distribution(site_decoded_counts)

# Plot
plt.bar(*zip(*raw_magnetization.items()), label="raw")
plt.bar(*zip(*decoded_magnetization.items()), label="decoded", width=0.3)
plt.legend()
plt.xlabel("Magnetization")
plt.ylabel("Frequency")
plt.title("Magnetization distribution with and without decoding")

Text(0.5, 1.0, 'Magnetization distribution with and without decoding')

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/8ead3582-16df-4616-836c-bdce867ad6b8-1.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/fd57d483-c70b-4ad5-b309-15750ad38bac-0.avif)

## Paso 2: Optimizar el problema para la ejecución en hardware cuántico
La transpilación de circuitos cuánticos para su ejecución en hardware generalmente involucra [varias etapas](/guides/transpiler-stages). Típicamente, las etapas que conllevan mayor costo computacional son la elección de la disposición de qubits, el enrutamiento de las compuertas de dos qubits para ajustarse a la conectividad del hardware, y la optimización del circuito para minimizar el conteo y la profundidad de compuertas. En el protocolo GEM, las etapas de disposición y enrutamiento son innecesarias porque la conectividad del hardware ya está incorporada en el diseño del protocolo. Los circuitos ya tienen una disposición de qubits, y las compuertas de dos qubits ya están mapeadas sobre conexiones nativas. Además, para preservar la estructura del circuito a medida que varía el ángulo $R_{ZZ}$, solo se debe realizar una optimización de circuito muy básica.

La clase `GemExperiment` transpila los circuitos de forma transparente al ejecutar el experimento. Las etapas de disposición y enrutamiento ya están anuladas por defecto para no hacer nada, y la optimización de circuitos se realiza a un nivel que solo optimiza compuertas de un solo qubit. Sin embargo, puede anular o pasar opciones adicionales utilizando el método `set_transpile_options`. Para fines de visualización, la siguiente celda de código transpila manualmente el circuito mostrado previamente y dibuja el circuito transpilado.

In [13]:
exp_data.figure("two_point_correlation")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/4ecb25c8-e572-49af-a879-9943039db131-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/e9b99d48-8d33-46b5-bff5-480ab1c1c1f2-0.avif)

## Paso 3: Ejecutar utilizando primitivas de Qiskit
Para ejecutar los circuitos del protocolo GEM en el hardware, llame al método `run` del objeto `GemExperiment`. Puede especificar el número de shots que desea muestrear de cada circuito. El método `run` devuelve un objeto [ExperimentData](https://qiskit-community.github.io/qiskit-experiments/stubs/qiskit_experiments.framework.ExperimentData.html) que debe guardar en una variable. Tenga en cuenta que el método `run` solo envía los trabajos sin esperar a que terminen, por lo que es una llamada no bloqueante.

In [14]:
exp_data.figure("normalized_variance")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/2b351d68-3924-445a-94ef-047b16214e8a-0.avif" alt="Output of the previous code cell" />

Para esperar los resultados, llame al método `block_for_results` del objeto `ExperimentData`. Esta llamada hará que el intérprete se detenga hasta que los trabajos hayan finalizado.

In [15]:
gem_exp = GemExperiment(
    plaquette_lattice.filter(range(3, 9)), backend=backend
)
gem_exp.set_experiment_options(schedule_idx=0)
exp_data = gem_exp.run(shots=10_000)
exp_data.block_for_results()
exp_data.figure("normalized_variance")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/08581c09-a6a5-4a56-9fc4-abf22b063c6a-0.avif" alt="Output of the previous code cell" />

In [16]:
gem_exp = GemExperiment(plaquette_lattice, backend=backend)
gem_exp.set_experiment_options(schedule_idx=0)
exp_data = gem_exp.run(shots=10_000)
exp_data.block_for_results()
exp_data.figure("normalized_variance")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/37e9a4cd-6efb-4ade-ad09-8139db9d58e9-0.avif" alt="Output of the previous code cell" />

## Paso 4: Posprocesar y devolver el resultado en el formato clásico deseado
A un ángulo $R_{ZZ}$ de $\frac{\pi}{2}$, el estado decodificado sería el estado GHZ en ausencia de ruido. El orden de largo alcance del estado GHZ se puede visualizar graficando la magnetización de las cadenas de bits medidas. La magnetización $M$ se define como la suma de los operadores de Pauli $Z$ de un solo qubit,
$$
M = \sum_{j=1}^N Z_j,
$$
donde $N$ es el número de qubits de sitio. Su valor para una cadena de bits es igual a la diferencia entre el número de ceros y el número de unos. Medir el estado GHZ produce el estado de todos ceros o el estado de todos unos con igual probabilidad, por lo que la magnetización sería $+N$ la mitad de las veces y $-N$ la otra mitad. En presencia de errores debidos al ruido, también aparecerían otros valores, pero si el ruido no es demasiado grande, la distribución seguiría estando concentrada cerca de $+N$ y $-N$.

Para las cadenas de bits sin procesar antes de la decodificación, la distribución de la magnetización sería equivalente a la de cadenas de bits uniformemente aleatorias, en ausencia de ruido.

La siguiente celda de código grafica la magnetización de las cadenas de bits sin procesar y las cadenas de bits decodificadas al ángulo $R_{ZZ}$ de $\frac{\pi}{2}$.